# Statistical Analysis of Current Commitments Dataset

This notebook performs comprehensive statistical analysis on current criminal commitments data, including:

- **Descriptive Statistics**: Overview of the dataset structure and basic metrics
- **Categorical Analysis**: Distribution of offense types, relationships, and prison status
- **Time-Based Analysis**: Temporal patterns in current offenses
- **Enhancement Analysis**: Patterns in sentence enhancements
- **Statistical Tests**: Chi-square, t-tests, correlation analysis, and Kruskal-Wallis tests

---

## Setup and Dependencies

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import chi2_contingency, ttest_ind, mannwhitneyu, kruskal
import warnings
warnings.filterwarnings('ignore')

# Set style for visualizations
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

print("Libraries loaded successfully!")

## Load Dataset

The dataset should be placed in the `data/` folder at the project root.

In [ ]:
# Load the dataset
# Adjust the path if running from a different directory
DATA_PATH = '../data/currentcommits.xlsx'

print("Loading Current Commitments dataset...")
df = pd.read_excel(DATA_PATH)

print(f"\nDataset loaded successfully!")
print(f"Total records: {len(df):,}")
print(f"Total columns: {len(df.columns)}")

---
## 1. Descriptive Statistics

Initial exploration of the dataset structure, data types, and basic statistics.

In [ ]:
# Dataset structure
print("Dataset Info:")
print(df.info())

In [ ]:
# Preview first few rows
print("First few rows:")
df.head()

In [ ]:
# Column names
print("Column names:")
print(df.columns.tolist())

In [ ]:
# Missing values analysis
print("Missing values per column:")
missing = df.isnull().sum()
missing_pct = (missing / len(df) * 100).round(2)
missing_df = pd.DataFrame({'Missing Count': missing, 'Missing %': missing_pct})
print(missing_df[missing_df['Missing Count'] > 0])

In [ ]:
# Basic statistics for numerical columns
print("Basic statistics for numerical columns:")
df.describe()

---
## 2. Categorical Variable Analysis

Examining the distribution of categorical variables in the dataset.

In [ ]:
# Define categorical columns to analyze
categorical_cols = ['sentencing county', 'offense', 'offense description', 
                    'offense category', 'in-prison', 'relationship']

for col in categorical_cols:
    if col in df.columns:
        print(f"\n{'='*50}")
        print(f"{col.upper()}")
        print(f"{'='*50}")
        print(f"\nTop 10 values:")
        print(df[col].value_counts().head(10))
        print(f"\nUnique values: {df[col].nunique()}")

---
## 3. Time-Based Analysis

Analyzing temporal patterns in the data.

In [ ]:
# Convert date columns to datetime
date_cols = ['offense begin date', 'offense end date']
for col in date_cols:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors='coerce')
        print(f"Converted '{col}' to datetime")

In [ ]:
# Extract year from offense begin date and analyze trends
if 'offense begin date' in df.columns:
    df['offense_year'] = df['offense begin date'].dt.year
    
    print("Current Offenses by Year (Last 20 years):")
    yearly_counts = df['offense_year'].value_counts().sort_index().tail(20)
    print(yearly_counts)
    
    # Visualization
    plt.figure(figsize=(14, 6))
    yearly_counts.plot(kind='bar', color='steelblue', edgecolor='black')
    plt.title('Number of Current Offenses by Year', fontsize=14)
    plt.xlabel('Year')
    plt.ylabel('Count')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

In [ ]:
# Calculate offense duration
if 'offense begin date' in df.columns and 'offense end date' in df.columns:
    df['offense_duration_days'] = (df['offense end date'] - df['offense begin date']).dt.days
    
    print("Offense Duration Statistics (in days):")
    print(df['offense_duration_days'].describe())

---
## 4. Enhancement Analysis

Analyzing sentence enhancements across records.

In [ ]:
# Count enhancements per record
enhancement_cols = [f'off_enh{i}' for i in range(1, 12)]
existing_enh_cols = [col for col in enhancement_cols if col in df.columns]

if existing_enh_cols:
    df['total_enhancements'] = df[existing_enh_cols].notna().sum(axis=1)
    
    print("Enhancement Statistics:")
    print(f"\nDistribution of enhancement counts:")
    print(df['total_enhancements'].value_counts().sort_index())
    print(f"\nAverage enhancements per record: {df['total_enhancements'].mean():.2f}")
    print(f"Median enhancements per record: {df['total_enhancements'].median():.2f}")
    print(f"Max enhancements on single record: {df['total_enhancements'].max():.0f}")
    
    # Visualization
    plt.figure(figsize=(10, 5))
    df['total_enhancements'].value_counts().sort_index().plot(kind='bar', color='darkred', edgecolor='black')
    plt.title('Distribution of Enhancements per Record', fontsize=14)
    plt.xlabel('Number of Enhancements')
    plt.ylabel('Count')
    plt.tight_layout()
    plt.show()

In [ ]:
# Most common enhancement types
print("Most Common Enhancement Types:")
for i in range(1, 4):  # Top 3 enhancement columns
    col = f'off_enh{i}'
    if col in df.columns:
        print(f"\n{col.upper()}:")
        print(df[col].value_counts().head(5))

---
## 5. Chi-Square Tests

Testing for associations between categorical variables using Chi-square tests of independence.

In [ ]:
# Test 1: Offense Category vs Relationship Type
if 'offense category' in df.columns and 'relationship' in df.columns:
    print("Chi-Square Test: Offense Category vs Relationship Type")
    print("="*60)
    
    contingency_table = pd.crosstab(df['offense category'], df['relationship'])
    chi2, p_value, dof, expected = chi2_contingency(contingency_table)
    
    print(f"\nChi-square statistic: {chi2:.4f}")
    print(f"P-value: {p_value:.4e}")
    print(f"Degrees of freedom: {dof}")
    print(f"\nResult: {'Statistically Significant' if p_value < 0.05 else 'Not Statistically Significant'} at α=0.05")
    
    print("\nContingency Table:")
    display(contingency_table)

In [ ]:
# Test 2: Offense Category vs In Prison Status
if 'offense category' in df.columns and 'in-prison' in df.columns:
    print("Chi-Square Test: Offense Category vs In-Prison Status")
    print("="*60)
    
    df_filtered = df[df['in-prison'].notna() & (df['in-prison'] != '')]
    
    if len(df_filtered) > 0:
        contingency_table2 = pd.crosstab(df_filtered['offense category'], df_filtered['in-prison'])
        chi2_2, p_value_2, dof_2, expected_2 = chi2_contingency(contingency_table2)
        
        print(f"\nChi-square statistic: {chi2_2:.4f}")
        print(f"P-value: {p_value_2:.4e}")
        print(f"Degrees of freedom: {dof_2}")
        print(f"\nResult: {'Statistically Significant' if p_value_2 < 0.05 else 'Not Statistically Significant'} at α=0.05")
        
        print("\nContingency Table:")
        display(contingency_table2)

In [ ]:
# Test 3: Has Enhancements vs Offense Category
if 'total_enhancements' in df.columns:
    print("Chi-Square Test: Has Enhancements vs Offense Category")
    print("="*60)
    
    df['has_enhancements'] = df['total_enhancements'] > 0
    contingency_table3 = pd.crosstab(df['offense category'], df['has_enhancements'])
    chi2_3, p_value_3, dof_3, expected_3 = chi2_contingency(contingency_table3)
    
    print(f"\nChi-square statistic: {chi2_3:.4f}")
    print(f"P-value: {p_value_3:.4e}")
    print(f"Degrees of freedom: {dof_3}")
    print(f"\nResult: {'Statistically Significant' if p_value_3 < 0.05 else 'Not Statistically Significant'} at α=0.05")
    
    print("\nContingency Table:")
    display(contingency_table3)

---
## 6. T-Tests

Comparing mean number of enhancements between different offense categories.

In [ ]:
# Compare number of enhancements between different offense categories
if 'total_enhancements' in df.columns and 'offense category' in df.columns:
    print("T-Test: Enhancements - Drug Crimes vs Property Crimes")
    print("="*60)
    
    drug_crimes = df[df['offense category'] == 'Drug Crimes']['total_enhancements'].dropna()
    property_crimes = df[df['offense category'] == 'Property Crimes']['total_enhancements'].dropna()
    
    if len(drug_crimes) > 0 and len(property_crimes) > 0:
        t_stat, p_value_t = ttest_ind(drug_crimes, property_crimes)
        
        print(f"\nDrug Crimes:")
        print(f"  - Sample size: {len(drug_crimes):,}")
        print(f"  - Mean enhancements: {drug_crimes.mean():.2f}")
        print(f"  - Std deviation: {drug_crimes.std():.2f}")
        
        print(f"\nProperty Crimes:")
        print(f"  - Sample size: {len(property_crimes):,}")
        print(f"  - Mean enhancements: {property_crimes.mean():.2f}")
        print(f"  - Std deviation: {property_crimes.std():.2f}")
        
        print(f"\nT-statistic: {t_stat:.4f}")
        print(f"P-value: {p_value_t:.4e}")
        print(f"\nResult: {'Statistically Significant difference' if p_value_t < 0.05 else 'No statistically significant difference'} at α=0.05")

In [ ]:
# T-Test: Crimes Against Persons vs Property Crimes
if 'total_enhancements' in df.columns and 'offense category' in df.columns:
    print("T-Test: Enhancements - Crimes Against Persons vs Property Crimes")
    print("="*60)
    
    person_crimes = df[df['offense category'] == 'Crimes Against Persons']['total_enhancements'].dropna()
    property_crimes = df[df['offense category'] == 'Property Crimes']['total_enhancements'].dropna()
    
    if len(person_crimes) > 0 and len(property_crimes) > 0:
        t_stat2, p_value_t2 = ttest_ind(person_crimes, property_crimes)
        
        print(f"\nCrimes Against Persons:")
        print(f"  - Sample size: {len(person_crimes):,}")
        print(f"  - Mean enhancements: {person_crimes.mean():.2f}")
        print(f"  - Std deviation: {person_crimes.std():.2f}")
        
        print(f"\nProperty Crimes:")
        print(f"  - Sample size: {len(property_crimes):,}")
        print(f"  - Mean enhancements: {property_crimes.mean():.2f}")
        print(f"  - Std deviation: {property_crimes.std():.2f}")
        
        print(f"\nT-statistic: {t_stat2:.4f}")
        print(f"P-value: {p_value_t2:.4e}")
        print(f"\nResult: {'Statistically Significant difference' if p_value_t2 < 0.05 else 'No statistically significant difference'} at α=0.05")

---
## 7. Correlation Analysis

Examining relationships between numerical variables.

In [ ]:
# Count current offenses per individual
if 'cdcno' in df.columns:
    current_counts = df.groupby('cdcno').size().reset_index(name='current_offense_count')
    
    print("Current Offense Count Statistics:")
    print(current_counts['current_offense_count'].describe())
    
    # Merge back to main dataframe
    df = df.merge(current_counts, on='cdcno', how='left')

In [ ]:
# Correlation between current offenses and enhancements
if 'current_offense_count' in df.columns and 'total_enhancements' in df.columns:
    print("Correlation Analysis: Current Offenses vs Enhancements")
    print("="*60)
    
    # Pearson correlation
    correlation = df[['current_offense_count', 'total_enhancements']].corr()
    print("\nPearson Correlation Matrix:")
    print(correlation)
    
    # Spearman correlation (non-parametric)
    df_corr = df[['current_offense_count', 'total_enhancements']].dropna()
    if len(df_corr) > 0:
        spearman_corr, spearman_p = stats.spearmanr(
            df_corr['current_offense_count'], 
            df_corr['total_enhancements']
        )
        
        print(f"\nSpearman Correlation: {spearman_corr:.4f}")
        print(f"P-value: {spearman_p:.4e}")
        print(f"\nResult: {'Statistically Significant correlation' if spearman_p < 0.05 else 'No statistically significant correlation'} at α=0.05")

---
## 8. Kruskal-Wallis Test

Non-parametric test to compare enhancements across multiple offense categories.

In [ ]:
if 'total_enhancements' in df.columns and 'offense category' in df.columns:
    print("Kruskal-Wallis Test: Enhancements across Offense Categories")
    print("="*60)
    
    # Get groups
    groups = []
    group_names = []
    categories = df['offense category'].dropna().unique()
    
    for cat in categories:
        group_data = df[df['offense category'] == cat]['total_enhancements'].dropna()
        if len(group_data) > 0:
            groups.append(group_data)
            group_names.append(cat)
    
    if len(groups) > 1:
        h_stat, p_value_kw = kruskal(*groups)
        
        print(f"\nNumber of groups: {len(groups)}")
        print(f"H-statistic: {h_stat:.4f}")
        print(f"P-value: {p_value_kw:.4e}")
        print(f"\nResult: {'Statistically Significant difference' if p_value_kw < 0.05 else 'No statistically significant difference'} at α=0.05")
        
        # Show mean enhancements by category
        print("\nMean Enhancements by Offense Category:")
        for cat in categories:
            cat_data = df[df['offense category'] == cat]['total_enhancements']
            print(f"  {cat}: {cat_data.mean():.2f} (n={len(cat_data):,})")

---
## 9. Distribution Analysis

Visualizing the distribution of key categorical variables.

In [ ]:
if 'offense category' in df.columns:
    print("Offense Category Distribution")
    print("="*60)
    
    offense_dist = df['offense category'].value_counts()
    offense_pct = (offense_dist / len(df) * 100).round(2)
    
    dist_df = pd.DataFrame({
        'Count': offense_dist,
        'Percentage': offense_pct
    })
    print(dist_df)
    
    # Visualization
    plt.figure(figsize=(12, 6))
    offense_dist.plot(kind='barh', color='steelblue', edgecolor='black')
    plt.title('Distribution of Offense Categories', fontsize=14)
    plt.xlabel('Count')
    plt.ylabel('Offense Category')
    plt.tight_layout()
    plt.show()

In [ ]:
if 'relationship' in df.columns:
    print("Relationship Type Distribution")
    print("="*60)
    
    relationship_dist = df['relationship'].value_counts()
    relationship_pct = (relationship_dist / len(df) * 100).round(2)
    
    dist_df = pd.DataFrame({
        'Count': relationship_dist,
        'Percentage': relationship_pct
    })
    print(dist_df)
    
    # Visualization
    plt.figure(figsize=(10, 6))
    relationship_dist.plot(kind='bar', color='coral', edgecolor='black')
    plt.title('Distribution of Relationship Types', fontsize=14)
    plt.xlabel('Relationship Type')
    plt.ylabel('Count')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

---
## 10. Current Commitment Patterns

Analyzing patterns among currently committed individuals.

In [ ]:
if 'cdcno' in df.columns:
    print("Current Commitment Patterns")
    print("="*60)
    
    commitment_data = df.groupby('cdcno').agg({
        'offense begin date': 'count',
        'total_enhancements': 'mean',
        'offense category': lambda x: x.mode()[0] if len(x.mode()) > 0 else None
    }).reset_index()
    commitment_data.columns = ['cdcno', 'total_current_offenses', 'avg_enhancements', 'most_common_offense']
    
    print(f"\nTotal unique individuals: {len(commitment_data):,}")
    print(f"Average current offenses per person: {commitment_data['total_current_offenses'].mean():.2f}")
    print(f"Median current offenses per person: {commitment_data['total_current_offenses'].median():.2f}")
    print(f"Average enhancements per person: {commitment_data['avg_enhancements'].mean():.2f}")
    
    print("\nDistribution of current offenses per person:")
    offense_per_person = commitment_data['total_current_offenses'].value_counts().sort_index().head(10)
    print(offense_per_person)
    
    # Visualization
    plt.figure(figsize=(12, 5))
    offense_per_person.plot(kind='bar', color='darkgreen', edgecolor='black')
    plt.title('Distribution of Current Offenses per Individual', fontsize=14)
    plt.xlabel('Number of Offenses')
    plt.ylabel('Number of Individuals')
    plt.tight_layout()
    plt.show()

---
## 11. Summary Report

Consolidated summary of key findings from the analysis.

In [ ]:
print("="*80)
print("SUMMARY REPORT")
print("="*80)

# Create summary dictionary
summary = {
    'Total Records': f"{len(df):,}",
    'Unique Individuals': f"{df['cdcno'].nunique():,}" if 'cdcno' in df.columns else 'N/A',
    'Date Range': f"{df['offense begin date'].min().strftime('%Y-%m-%d')} to {df['offense begin date'].max().strftime('%Y-%m-%d')}" if 'offense begin date' in df.columns and df['offense begin date'].notna().any() else 'N/A',
    'Most Common Offense Category': df['offense category'].mode()[0] if 'offense category' in df.columns else 'N/A',
    'Average Enhancements per Record': f"{df['total_enhancements'].mean():.2f}" if 'total_enhancements' in df.columns else 'N/A',
    'Records with Enhancements': f"{(df['has_enhancements'].sum() / len(df) * 100):.2f}%" if 'has_enhancements' in df.columns else 'N/A'
}

for key, value in summary.items():
    print(f"{key}: {value}")

print("\n" + "="*80)
print("ANALYSIS COMPLETE")
print("="*80)